In [11]:
import cv2
import tempfile
import imutils
import numpy as np
from PIL import Image
from matplotlib.pyplot import imshow

%matplotlib inline

In [14]:
image = cv2.imread('test.jpg',1)
#image = image.astype('uint8')

In [35]:
image.shape

(937, 1158, 3)

In [15]:
# this piece of code works if the text, say, located on the white that lies on the black table
# otherwise screenCnt remains None
# on the input the colord picture is given

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (5, 5), 0)
edged = cv2.Canny(gray, 10, 50)
cnts = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if imutils.is_cv2() else cnts[1]
cnts = sorted(cnts, key=cv2.contourArea, reverse=True)[:5]
#print(cnts.shape)

screenCnt = None
for c in cnts:
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.02 * peri, True)
    if len(approx) == 4:
        screenCnt = approx
        print('break')
        break

im1 = Image.open(r'test_cv.jpg', mode='r')
#im1 = Image.fromarray(edged)
im1 = Image.fromarray(cv2.drawContours(image, [screenCnt], -1, (0, 255, 0), 2))
im1.save('test_cv.jpg')

break


In [17]:
# one batch of funcitons to rotate the image with any rotation

def order_points(pts):
    # initialzie a list of coordinates that will be ordered
    # such that the first entry in the list is the top-left,
    # the second entry is the top-right, the third is the
    # bottom-right, and the fourth is the bottom-left
    rect = np.zeros((4, 2), dtype="float32")

    # the top-left point will have the smallest sum, whereas
    # the bottom-right point will have the largest sum
    s = pts.sum(axis=1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]

    # now, compute the difference between the points, the
    # top-right point will have the smallest difference,
    # whereas the bottom-left will have the largest difference
    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]

    # return the ordered coordinates
    return rect


def four_point_transform(image, pts):
    # obtain a consistent order of the points and unpack them
    # individually
    rect = order_points(pts)
    (tl, tr, br, bl) = rect

    # compute the width of the new image, which will be the
    # maximum distance between bottom-right and bottom-left
    # x-coordiates or the top-right and top-left x-coordinates
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))

    # compute the height of the new image, which will be the
    # maximum distance between the top-right and bottom-right
    # y-coordinates or the top-left and bottom-left y-coordinates
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))

    # now that we have the dimensions of the new image, construct
    # the set of destination points to obtain a "birds eye view",
    # (i.e. top-down view) of the image, again specifying points
    # in the top-left, top-right, bottom-right, and bottom-left
    # order
    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype="float32")

    # compute the perspective transform matrix and then apply it
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
    return warped

In [7]:
# second approach to rotate an image
# doesn't work correctly. Possibly, it works with angels multiples 45 
def rotate(img):
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray_img = cv2.bitwise_not(gray_img)
    coordinates = np.column_stack(np.where(gray_img > 0))
    ang = cv2.minAreaRect(coordinates)[-1]
    print(ang)
    if ang < -45: 
        ang = -(90 + ang)
    else:
        ang = -ang
    height, width = img.shape[:2]

    center_img = (width / 2, height / 2)

    rotationMatrix = cv2.getRotationMatrix2D(center_img, ang, 1.0)

    rotated_img = cv2.warpAffine(img, rotationMatrix, (width, height), borderMode = cv2.BORDER_REFLECT)
    return rotated_img

In [10]:
rotated = rotate(image)

-90.0


In [6]:
im1 = Image.open(r'test_cv.jpg', mode='r')
im1 = Image.fromarray(rotated)
im1.save('test_cv.jpg')

In [18]:
def remove_noise_and_smooth(img):
    # input should be grayscale image
    filtered = cv2.adaptiveThreshold(img.astype(np.uint8), 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 9, 41)
    kernel = np.ones((1, 1), np.uint8)
    opening = cv2.morphologyEx(filtered, cv2.MORPH_OPEN, kernel)
    closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel)
    # bluring (it can be as you wish. See the doc)
    img = cv2.GaussianBlur(img,(3,3),0)
    or_image = cv2.bitwise_or(img, closing)
    return or_image

In [19]:
# rotate image
ratio = 1
pts = np.array(screenCnt.reshape(4, 2) * ratio)
warped = four_point_transform(image, pts)



In [ ]:
im1 = Image.open(r'test_cv.jpg', mode='r')
im1 = Image.fromarray(warped)
im1.save('test_cv.jpg')

In [20]:
removed_noise = remove_noise_and_smooth(cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY))

In [34]:
# sharp the image
kernel_sharpening = np.array([[-1,-1,-1], 
                              [-1, 8,-1],
                              [-1,-1,-1]])

sharpened = cv2.filter2D(smooth, -1, kernel_sharpening)

In [26]:
# another sharp (or not)
blur = cv2.GaussianBlur(removed_noise,(3,3),0)
smooth = cv2.addWeighted(blur,1.5,removed_noise,-0.5,0)

In [35]:
im1 = Image.open(r'test_cv.jpg', mode='r')
im1 = Image.fromarray(sharpened)
im1.save('test_cv.jpg')

In [ ]:
cv2.imshow('Original Image', remove_noise_and_smooth(cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY))) 
cv2.waitKey(0)

# Sources

[\[1\]](https://answers.opencv.org/question/121205/how-to-refine-the-edges-of-an-image/)

[\[2\]](https://medium.com/cashify-engineering/improve-accuracy-of-ocr-using-image-preprocessing-8df29ec3a033)